# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [25]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [26]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,atuona,-9.8000,-139.0333,26.13,76,22,7.43,P,1675829422
1,1,namatanai,-3.6667,152.4333,29.76,64,90,4.76,P,1675829423
2,2,hailar,49.2000,119.7000,-18.33,94,96,1.38,C,1675829423
3,3,nanortalik,60.1432,-45.2371,-15.45,75,11,7.06,G,1675829424
4,4,porto novo,6.4965,2.6036,25.24,86,96,1.89,B,1675829425


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map plot
map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

In [28]:
%%capture --no-display

# Configure the map plot
# base_url = "https://api.geoapify.com/v2/places"

# Display the map
# YOUR CODE HERE

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [29]:
# Narrow down cities that fit criteria and drop any results with null values
city_special=city_data_df[(city_data_df['Lat'].between(-20,20))&(city_data_df['Lng'].between(50, 200))]
city_special
# Drop any rows with null values 


city_special.dropna()

# Display sample data
city_special


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,namatanai,-3.6667,152.4333,29.76,64,90,4.76,P,1675829423
15,15,ugoofaaru,5.6667,73.0000,27.04,67,22,2.73,M,1675829338
21,21,sulangan,11.1373,123.7228,26.63,75,100,5.41,P,1675829437
50,50,vanimo,-2.6741,141.3028,25.43,94,100,4.05,P,1675829447
81,81,aitape,-3.1374,142.3475,25.38,93,100,4.62,P,1675829457
...,...,...,...,...,...,...,...,...,...,...
502,502,palauig,15.4350,119.9045,28.69,66,100,3.04,P,1675829696
515,515,bontang,0.1333,117.5000,25.73,74,99,1.93,I,1675829700
517,517,san policarpo,12.1791,125.5072,27.61,70,100,7.10,P,1675829705
560,560,majene,-3.5403,118.9707,31.34,56,66,3.86,I,1675829751


### Step 3: Create a new DataFrame called `hotel_df`.

In [30]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_special.copy(deep=True)

# Display sample data
hotel_df=hotel_df.drop(columns=['City_ID', 'Max Temp', 'Cloudiness', 'Wind Speed', 'Date'])


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=''
hotel_df.head()

# Display sample data
# YOUR CODE HERE

,City,Lat,Lng,Humidity,Country,Hotel Name
1,namatanai,-3.6667,152.4333,64,P,
15,ugoofaaru,5.6667,73.0000,67,M,
21,sulangan,11.1373,123.7228,75,P,
50,vanimo,-2.6741,141.3028,94,P,
81,aitape,-3.1374,142.3475,93,P,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [31]:
# Set parameters to search for a hotel
radius = 10000
categories='accommodation.hotel'

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
# get latitude, longitude from the DataFrame
    longitude=row['Lng']
    latitude=row['Lat'] 
   
    # Add filter and bias parameters with the current city's latitude and longitude to the params 
    # dictionary
    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 1
    
    
    params = {
    "categories": categories,
    "limit":limit,
    "filter":filters,
    "bias":bias,
    "apiKey":geoapify_key
    }
        
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"
    
     # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params).json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df
     
   

Starting hotel search
namatanai - nearest hotel: Namatanai Hotel
ugoofaaru - nearest hotel: No hotel found
sulangan - nearest hotel: Marion's Inn
vanimo - nearest hotel: CBC Guesthouse
aitape - nearest hotel: No hotel found
kathu - nearest hotel: Tinidee Hotel
padang - nearest hotel: losmen Surya
hithadhoo - nearest hotel: Pebbles Inn
culaba - nearest hotel: No hotel found
metro - nearest hotel: Hotel Nuban
butaritari - nearest hotel: No hotel found
georgetown - nearest hotel: Page 63 hostel
quang ngai - nearest hotel: Ngoc Anh
thinadhoo - nearest hotel: No hotel found
palu - nearest hotel: Ujung Pandang Hotel
manokwari - nearest hotel: Hotel Arfak
isangel - nearest hotel: Tanna Lodge
eydhafushi - nearest hotel: The Westin Maldives Miriandhoo Resort
airai - nearest hotel: No hotel found
larap - nearest hotel: Hotel Bienvinido
innisfail - nearest hotel: No hotel found
labuhan - nearest hotel: No hotel found
dingle - nearest hotel: Hotel Edmundo
balikpapan - nearest hotel: Hotel Grand Se

,City,Lat,Lng,Humidity,Country,Hotel Name
1,namatanai,-3.6667,152.4333,64,P,Namatanai Hotel
15,ugoofaaru,5.6667,73.0000,67,M,No hotel found
21,sulangan,11.1373,123.7228,75,P,Marion's Inn
50,vanimo,-2.6741,141.3028,94,P,CBC Guesthouse
81,aitape,-3.1374,142.3475,93,P,No hotel found
...,...,...,...,...,...,...
502,palauig,15.4350,119.9045,66,P,Armada Resort
515,bontang,0.1333,117.5000,74,I,Hotel Sederhana
517,san policarpo,12.1791,125.5072,70,P,RJV hôtel
560,majene,-3.5403,118.9707,56,I,No hotel found


In [32]:
hotel_df=hotel_df[hotel_df['Hotel Name']!="No hotel found"]
hotel_df.head()

,City,Lat,Lng,Humidity,Country,Hotel Name
1,namatanai,-3.6667,152.4333,64,P,Namatanai Hotel
21,sulangan,11.1373,123.7228,75,P,Marion's Inn
50,vanimo,-2.6741,141.3028,94,P,CBC Guesthouse
82,kathu,7.9178,98.3332,66,T,Tinidee Hotel
85,padang,-0.9492,100.3543,58,I,losmen Surya


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [33]:
%%capture --no-display

# Configure the map plot
map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 0.8,
    color = "Hotel Name"
)

# Display the map plot
map



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Humidity)